In [221]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Dense,
    Concatenate,
    Input,
    Embedding,
    Lambda,
    TextVectorization,
    Normalization,
    GlobalAveragePooling2D,
    GlobalAveragePooling1D,
)
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from sklearn.model_selection import KFold
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import ast

In [222]:
user_df = pd.read_csv("../dataset/users.csv", delimiter=";")
artwork_df = pd.read_csv("../dataset/artworks.csv", delimiter=";")
artwork_df = artwork_df.drop_duplicates(subset=["title", "author"], keep="first")
artwork_df["tag_string"] = artwork_df["image_tags"].apply(
    lambda x: " ".join(ast.literal_eval(x))
)

In [223]:
tfidf_vectorizer = TfidfVectorizer()
tag_vectors = tfidf_vectorizer.fit_transform(artwork_df["tag_string"])

In [224]:
user_df["place"] = user_df["place"].astype(str)
user_df["place"] = user_df["place"].fillna("N/A")

user_df["inscription_date"] = user_df["inscription_date"].astype(str)
user_df["inscription_date"] = user_df["inscription_date"].fillna("N/A")

In [225]:
def preprocess_image(image_url):
    response = requests.get(image_url)
    if response.status_code == 200:
        image = Image.open(BytesIO(response.content))
        image = image.convert("RGB")
        image = image.resize((224, 224))
        image_array = np.array(image)
        image_array = image_array / 255.0
        return image_array
    else:
        raise FileNotFoundError(
            f"Impossibile scaricare l'immagine dall'URL: {image_url}"
        )

In [278]:
def get_similar_image(user_artworks, artwork_df, tag_vectors):
    user_tags_string = " ".join(user_artworks["tag_string"].values)
    user_tags_vector = tfidf_vectorizer.transform([user_tags_string])
    similarity = cosine_similarity(user_tags_vector, tag_vectors)
    average_similarity = similarity.mean(axis=0)
    similar_image_index = average_similarity.argmax()
    return pd.DataFrame([artwork_df.iloc[similar_image_index]])

In [255]:
def create_triples(user_df, artwork_df):
    triples = []
    for _, user_row in user_df.iterrows():
        user_interactions = artwork_df[artwork_df["author"] == user_row["name"]]
        Pu = user_interactions.sample(frac=1)
        i = get_similar_image(user_interactions, artwork_df, tag_vectors)
        non_interacted = artwork_df[~artwork_df["author"].isin([user_row["name"]])]
        j = non_interacted.sample()
        triples.append((Pu, i, j))
    return triples

In [258]:
def preprocess_data(triples, image_preprocessor):
    processed_triples = []

    for Pu, i, j in triples:
        Pu_images = np.array([image_preprocessor(url) for url in Pu["img"].values])
        i_image = np.array(image_preprocessor(i["img"].values[0]))
        j_image = np.array(image_preprocessor(j["img"].values[0]))
        processed_triples.append((Pu_images, i_image, j_image, Pu, i.iloc[0], j.iloc[0]))

    return processed_triples

In [229]:
embedding_dim = 200
pu_dim = 400
max_text_words = 5000
max_comment_words = 1000

In [230]:
name_vectorizer = TextVectorization(max_tokens=max_text_words)
name_vectorizer.adapt(user_df["name"])
place_vectorizer = TextVectorization(max_tokens=max_text_words)
place_vectorizer.adapt(user_df["place"])

inscription_date_vectorizer = TextVectorization(max_tokens=max_text_words)
inscription_date_vectorizer.adapt(user_df["inscription_date"])

art_title_vectorizer = TextVectorization(max_tokens=max_text_words)
art_title_vectorizer.adapt(artwork_df["title"])

art_author_vectorizer = TextVectorization(max_tokens=max_text_words)
art_author_vectorizer.adapt(artwork_df["author"])

art_date_vectorizer = TextVectorization(max_tokens=max_text_words)
art_date_vectorizer.adapt(artwork_df["date"])


In [231]:
resnet = ResNet50(weights='imagenet', include_top=False, pooling=None)
for layer in resnet.layers:
    layer.trainable = False
def extract_features(image):
    features=resnet(image)
    features = GlobalAveragePooling2D()(features)
    return features

In [232]:
def create_text_embedding(text_input, max_words, output_dim):
    embedding = Embedding(input_dim=max_words, output_dim=output_dim)(text_input)
    pooling_embedding = GlobalAveragePooling1D()(embedding)
    return pooling_embedding

In [233]:
normalization_layer = Normalization()

In [234]:
input_user_name = Input(shape=(1,), name="input_user_name")
input_user_place = Input(shape=(1,), name="input_user_place")
input_user_inscription_date = Input(shape=(1,), name="input_user_inscription_date")
input_user_page_views = Input(shape=(1,), name="input_user_page_views")
input_user_followers = Input(shape=(1,), name="input_user_followers")
input_user_follow = Input(shape=(1,), name="input_user_follow")
input_user_favourites = Input(shape=(1,), name="input_user_favourites")
input_user_comments_made = Input(shape=(1,), name="input_user_comments_made")
input_user_comments_received = Input(shape=(1,), name="input_user_comments_received")

normalized_inscription_date = normalization_layer(input_user_inscription_date)
normalized_page_views = normalization_layer(input_user_page_views)
normalized_followers = normalization_layer(input_user_followers)
normalized_follow = normalization_layer(input_user_follow)
normalized_favourites = normalization_layer(input_user_favourites)
normalized_comments_made = normalization_layer(input_user_comments_made)
normalized_comments_received = normalization_layer(input_user_comments_received)

normalized_inscription_date = Dense(embedding_dim, activation='selu')(normalized_inscription_date)
normalized_page_views = Dense(embedding_dim, activation="selu")(normalized_page_views)
normalized_followers = Dense(embedding_dim, activation="selu")(normalized_followers)
normalized_follow = Dense(embedding_dim, activation="selu")(normalized_follow)
normalized_favourites = Dense(embedding_dim, activation="selu")(normalized_favourites)
normalized_comments_made = Dense(embedding_dim, activation="selu")(
    normalized_comments_made
)
normalized_comments_received = Dense(embedding_dim, activation="selu")(
    normalized_comments_received
)

user_features = Concatenate(name="user_features")(
    [
        create_text_embedding(input_user_name, max_text_words, embedding_dim),
        create_text_embedding(input_user_place, max_text_words, embedding_dim),
        create_text_embedding(input_user_inscription_date, max_text_words, embedding_dim),
        normalized_page_views,
        normalized_followers,
        normalized_follow,
        normalized_favourites,
        normalized_comments_made,
        normalized_comments_received,
    ]
)

In [235]:
input_art_title = Input(shape=(14,), name="input_art_title")
input_art_author = Input(shape=(1,), name="input_art_author")
input_art_likes = Input(shape=(1,), name="input_art_likes")
input_art_views = Input(shape=(1,), name="input_art_views")
input_art_date = Input(shape=(1,), name="input_art_date")

art_title_embedding = create_text_embedding(
    input_art_title, max_text_words, embedding_dim
)
art_author_embedding = create_text_embedding(
    input_art_author, max_text_words, embedding_dim
)
art_date_embedding = create_text_embedding(
    input_art_date, max_text_words, embedding_dim
)

art_features = Concatenate(name="art_features")(
    [
        art_title_embedding,
        art_author_embedding,
        normalization_layer(input_art_likes),
        normalization_layer(input_art_views),
        normalization_layer(input_art_date),
    ]
)

In [236]:
def custom_reduce_sum(x, y):
    return K.sum(x * y, axis=1)

In [237]:
def custom_triplet_loss(y_true, y_pred, margin=0.4):
    score_i, score_j = tf.split(y_pred, num_or_size_splits=2, axis=-1)
    loss = tf.maximum(0.0, margin + score_j - score_i)
    return tf.reduce_mean(loss)

In [238]:
input_pu = Input(shape=(3,224, 224, 3), name="input_pu")
input_i = Input(shape=(224, 224, 3), name="input_i")
input_j = Input(shape=(224, 224, 3), name="input_j")
pu_features = Lambda(
    lambda x: K.map_fn(
        lambda y: extract_features(y), x
    ),
    output_shape=(None, embedding_dim),
)(input_pu)

i_features = extract_features(input_i)
j_features = extract_features(input_j)

dense_layer_1 = Dense(embedding_dim, activation="selu", name="dense_layer_1")
dense_layer_2 = Dense(embedding_dim, activation="selu", name="dense_layer_2")

reduced_pu = Lambda(
    lambda x: K.map_fn(
        lambda y: dense_layer_2(dense_layer_1(y)), x
    ),
    output_shape=(None, embedding_dim),
)(pu_features)


reduced_i = dense_layer_2(dense_layer_1(i_features))
reduced_j = dense_layer_2(dense_layer_1(j_features))

reduced_user_features = Dense(300, activation="relu", name="user_embedding")(
    user_features
)
reduced_art_features = Dense(300, activation="relu", name="artwork_embedding")(
    art_features
)

concat_i = Concatenate()([reduced_i, reduced_user_features, reduced_art_features])
concat_j = Concatenate()([reduced_j, reduced_user_features, reduced_art_features])


dense_comb_i= Dense(embedding_dim, activation="selu", name="dense_comb_i")(concat_i)
dense_comb_j= Dense(embedding_dim, activation="selu", name="dense_comb_j")(concat_j)


average_pooled_pu = Lambda(lambda x: K.mean(x, axis=1), output_shape=(embedding_dim,))(
    reduced_pu
)
max_pooled_pu = Lambda(lambda x: K.max(x, axis=1), output_shape=(embedding_dim,))(
    reduced_pu
)

pooled_pu = Concatenate()([average_pooled_pu, max_pooled_pu])


pu_dense_1 = Dense(300, activation="selu", name="pu_dense_1")(pooled_pu)
pu_dense_2 = Dense(200, activation="selu", name="pu_dense_2")(pu_dense_1)

final_pu = Dense(200, activation="selu", name="pu_dense_3")(pu_dense_2)

In [239]:
score_i = Lambda(lambda x: K.sum(x[0] * x[1], axis=1, keepdims=True))([final_pu, dense_comb_i])
score_j = Lambda(lambda x: K.sum(x[0] * x[1], axis=1, keepdims=True))([final_pu, dense_comb_j])
output_scores = Concatenate(axis=-1)([score_i, score_j])

In [240]:

def build_curatornet_model():
    tf.keras.backend.clear_session()
    curatornet = Model(
        inputs=[
            input_user_name,
            input_user_place,
            input_user_inscription_date,
            input_user_page_views,
            input_user_followers,
            input_user_follow,
            input_user_favourites,
            input_user_comments_made,
            input_user_comments_received,
            input_art_title,
            input_art_author,
            input_art_likes,
            input_art_views,
            input_art_date,
            input_pu,
            input_i,
            input_j,
        ],
        outputs=output_scores,
    )
    return curatornet

# curatornet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.00001), loss=custom_triplet_loss)
# curatornet.summary()

In [256]:
def prepare_inputs(processed_triples, users_df, target_image_count=3, expected_length=379):
    inputs = {
        "input_user_name": [],
        "input_user_place": [],
        "input_user_inscription_date": [],
        "input_user_page_views": [],
        "input_user_followers": [],
        "input_user_follow": [],
        "input_user_favourites": [],
        "input_user_comments_made": [],
        "input_user_comments_received": [],
        "input_art_title": [],
        "input_art_author": [],
        "input_art_likes": [],
        "input_art_views": [],
        "input_art_date": [],
        "input_pu": [],
        "input_i": [],
        "input_j": [],
    }

    for pu_images, i_image, j_image, Pu_meta, i_meta, j_meta in processed_triples:
        user_features = users_df[users_df["name"] == i_meta["author"]].iloc[0]
        if pu_images.shape[0] < target_image_count:
            pad_width = target_image_count - pu_images.shape[0]
            pu_images = np.pad(pu_images, ((0, pad_width), (0, 0), (0, 0), (0, 0)), mode='constant')
        elif pu_images.shape[0] > target_image_count:
            pu_images = pu_images[:target_image_count]

        inputs["input_pu"].append(pu_images)
        inputs["input_i"].append(i_image)
        inputs["input_j"].append(j_image)
        inputs["input_user_name"].append(user_features["name"])
        inputs["input_user_place"].append(user_features["place"])
        inputs["input_user_inscription_date"].append(user_features["inscription_date"])
        inputs["input_user_page_views"].append(user_features["number_page_views"])
        inputs["input_user_followers"].append(user_features["number_followers"])
        inputs["input_user_follow"].append(user_features["number_follow"])
        inputs["input_user_favourites"].append(user_features["number_favourites"])
        inputs["input_user_comments_made"].append(user_features["number_comments_made"])
        inputs["input_user_comments_received"].append(
            user_features["number_comments_receveid"]
        )
        inputs["input_art_title"].append(i_meta["title"])
        inputs["input_art_author"].append(i_meta["author"])
        inputs["input_art_likes"].append(i_meta["likes"])
        inputs["input_art_views"].append(i_meta["number_of_views"])
        inputs["input_art_date"].append(i_meta["date"])

    for key in inputs:
        try:
            if not isinstance(inputs[key], np.ndarray):
                inputs[key] = np.array(inputs[key])

            current_length = len(inputs[key])
            if current_length != expected_length:
                print(f"Adjusting {key} from {current_length} to {expected_length}")
                repeat_factor = max(1, expected_length // current_length)
                if repeat_factor > 1:
                    inputs[key] = np.tile(inputs[key], (repeat_factor,) + (1,) * (inputs[key].ndim - 1))
                if len(inputs[key]) > expected_length:
                    inputs[key] = inputs[key][:expected_length]
        except Exception as e:
            print(f"Warning: Could not convert {key} to numpy array: {e}")
    return inputs

In [242]:
def vectorizer_input(inputs):
  inputs["input_user_name"] = name_vectorizer(inputs["input_user_name"])
  inputs["input_user_place"] = place_vectorizer(inputs["input_user_place"])
  inputs["input_user_inscription_date"] = inscription_date_vectorizer(inputs["input_user_inscription_date"])
  inputs["input_art_title"] = art_title_vectorizer(inputs["input_art_title"])
  inputs["input_art_author"] = art_author_vectorizer(inputs["input_art_author"])
  inputs["input_art_date"] = art_date_vectorizer(inputs["input_art_date"])
  return inputs


In [243]:
def expand_inputs(inputs):
  inputs["input_user_page_views"] = np.expand_dims(inputs["input_user_page_views"], axis=-1)
  inputs["input_user_followers"] = np.expand_dims(inputs["input_user_followers"], axis=-1)
  inputs["input_user_follow"] = np.expand_dims(inputs["input_user_follow"], axis=-1)
  inputs["input_user_favourites"] = np.expand_dims(inputs["input_user_favourites"], axis=-1)
  inputs["input_user_comments_made"] = np.expand_dims(inputs["input_user_comments_made"], axis=-1)
  inputs["input_user_comments_received"] = np.expand_dims(inputs["input_user_comments_received"], axis=-1)
  inputs["input_art_likes"] = np.expand_dims(inputs["input_art_likes"], axis=-1)
  inputs["input_art_views"] = np.expand_dims(inputs["input_art_views"], axis=-1)
  return inputs

In [244]:
def replace_nans_in_inputs(inputs):
    for key, value in inputs.items():
        if isinstance(value, np.ndarray):
            value = tf.convert_to_tensor(value)
        if value.dtype.is_floating:
            if tf.math.reduce_any(tf.math.is_nan(value)):
                inputs[key] = tf.where(tf.math.is_nan(value), tf.zeros_like(value), value)
            else:
                print(f"Nessun NaN in {key}")
        else:
            print(f"{key} non è un tipo a virgola mobile, saltato.")       

In [245]:
def convert_inputs_to_tensors(inputs):
    for key, value in inputs.items():
        if isinstance(value, np.ndarray) or isinstance(value, list):
            inputs[key] = tf.convert_to_tensor(value)
    return inputs

In [260]:
triples = create_triples(user_df, artwork_df)
preprocessed_triples = preprocess_data(triples, preprocess_image)
filtered_triples = [triple for triple in preprocessed_triples if triple[0].ndim == 4]
inputs = prepare_inputs(filtered_triples, user_df)
inputs = vectorizer_input(inputs)
inputs = expand_inputs(inputs)
convert_inputs_to_tensors(inputs)
replace_nans_in_inputs(inputs)

processate triple
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224,

In [261]:
X_train, X_test = {}, {}
for key, value in inputs.items():
    print(f"Splitting {key}")
    indices = tf.range(start=0, limit=tf.shape(value)[0], dtype=tf.int32)
    train_idx, test_idx = train_test_split(indices.numpy(), test_size=0.2, random_state=42) 
    train_idx = tf.convert_to_tensor(train_idx, dtype=tf.int32)
    test_idx = tf.convert_to_tensor(test_idx, dtype=tf.int32)
    X_train[key] = tf.gather(value, train_idx)
    X_test[key] = tf.gather(value, test_idx)
    print(f"Key: {key}, dtype: {value.dtype}, train shape: {X_train[key].shape}, test shape: {X_test[key].shape}")

Splitting input_user_name
Key: input_user_name, dtype: <dtype: 'int64'>, train shape: (303, 1), test shape: (76, 1)
Splitting input_user_place
Key: input_user_place, dtype: <dtype: 'int64'>, train shape: (303, 1), test shape: (76, 1)
Splitting input_user_inscription_date
Key: input_user_inscription_date, dtype: <dtype: 'int64'>, train shape: (303, 1), test shape: (76, 1)
Splitting input_user_page_views
Key: input_user_page_views, dtype: <dtype: 'float64'>, train shape: (303, 1), test shape: (76, 1)
Splitting input_user_followers
Key: input_user_followers, dtype: <dtype: 'float64'>, train shape: (303, 1), test shape: (76, 1)
Splitting input_user_follow
Key: input_user_follow, dtype: <dtype: 'float64'>, train shape: (303, 1), test shape: (76, 1)
Splitting input_user_favourites
Key: input_user_favourites, dtype: <dtype: 'float64'>, train shape: (303, 1), test shape: (76, 1)
Splitting input_user_comments_made
Key: input_user_comments_made, dtype: <dtype: 'float64'>, train shape: (303, 1), 

In [262]:
labels = np.zeros((len(triples), 1))
train_labels = labels[:303]
test_labels = labels[303 : 303 + 76]
curatornet_model = build_curatornet_model()
curatornet_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.000001), loss=custom_triplet_loss, metrics=["accuracy"])
curatornet_model.fit(X_train, train_labels, epochs=10, batch_size=32)
val_loss, val_acc = curatornet_model.evaluate(X_test, test_labels)
print(f"Validation loss: {val_loss}, Validation accuracy: {val_acc}")

Modello creato
Modello compilato
Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 92s 6s/step - accuracy: 9.1572e-04 - loss: 132184.3281
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 65s 7s/step - accuracy: 0.0027 - loss: 111325.4062
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 65s 6s/step - accuracy: 0.0142 - loss: 89334.6719
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 68s 7s/step - accuracy: 0.1298 - loss: 65926.3125
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 71s 7s/step - accuracy: 0.4350 - loss: 62719.5430
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 73s 7s/step - accuracy: 0.5547 - loss: 51532.9141
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 71s 7s/step - accuracy: 0.5926 - loss: 45909.9766
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 71s 7s/step - accuracy: 0.6716 - loss: 32659.3203
Epoch 9/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 72s 7s/step - accuracy: 0.7370 - loss: 23444.1191
Epoch 10/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 72s 7s/step - accuracy: 0.6838 - loss: 24581.9844
Modello addestrato


ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76
'y' sizes: 667


In [270]:
embedding_model = Model(
    inputs=curatornet_model.input,
    outputs=[
        curatornet_model.get_layer("user_embedding").output,
        curatornet_model.get_layer("artwork_embedding").output,
    ],
)
user_embeddings, art_embeddings = embedding_model.predict(inputs)

12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step


In [272]:
def precision_at_k(actual, predicted, k):
    actual_set = set(actual)
    predicted_at_k = predicted[:k]
    return len(set(predicted_at_k) & actual_set) / k


def recall_at_k(actual, predicted, k):
    actual_set = set(actual)
    predicted_at_k = predicted[:k]
    return len(set(predicted_at_k) & actual_set) / len(actual_set)


def ndcg_at_k(actual, predicted, k):
    actual_set = set(actual)
    predicted_at_k = predicted[:k]
    dcg = sum(
        [
            1.0 / np.log2(i + 2) if predicted_at_k[i] in actual_set else 0.0
            for i in range(k)
        ]
    )
    idcg = sum([1.0 / np.log2(i + 2) for i in range(min(len(actual), k))])
    return dcg / idcg if idcg > 0 else 0.0

In [273]:
def get_top_k_recommendations(user_embedding, artwork_embeddings, k=10):
    similarities = np.dot(artwork_embeddings, user_embedding)
    top_k_indices = np.argsort(similarities)[::-1][:k]
    return top_k_indices

In [274]:
def top_k_artworks_by_popularity_score(k):
    cleaned_artworks_df = artwork_df.sort_values(
        "number_of_views", ascending=False
    ).drop_duplicates(subset=["title", "author"], keep="first")
    cleaned_artworks_df["popularity_score"] = (
        cleaned_artworks_df["number_of_views"] * 0.2  
        + cleaned_artworks_df["likes"] * 0.5  
        + cleaned_artworks_df["number_of_comments"] * 0.3  
    )
    top_k_artworks = cleaned_artworks_df.nlargest(k, "popularity_score")
    return top_k_artworks.index.tolist()

In [277]:
test = get_top_k_recommendations(user_embeddings[0], art_embeddings, k=150)
views_set = top_k_artworks_by_popularity_score(200)
precision = precision_at_k(views_set, test, 150)
recall = recall_at_k(views_set, test, 150)
ndcg = ndcg_at_k(views_set, test, 150)
print(f"Precision: {precision}, Recall: {recall}, NDCG: {ndcg}")

Precision: 0.18666666666666668, Recall: 0.14, NDCG: 0.1706561175512317
